In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests


personal_data=requests.get('https://api.bjs.ojp.gov/bjs/ncvs/v2/personal/2018?format=json').json()
household_data=requests.get('https://api.bjs.ojp.gov/bjs/ncvs/v2/household/2018?format=json').json()


In [19]:
url = "https://api.bjs.ojp.gov/bjs/ncvs/v2/personal/"

years = ["2014", "2015", "2016", "2017", "2018"]

# set up lists to hold data
year = []
income = []
gender = []


# # Loop through the list of cities and perform a request for data on each (Personal)
# for year in years:
    
#     try:
#         response = requests.get(f'https://api.bjs.ojp.gov/bjs/ncvs/v2/personal/{year}?format=json').json()
#         year.append(response)
#         income.append(response)
#         gender.append(response['personalData'][0]['gender'])
# #         print(f"The year is {year}")

# #         print(f"The gender is {gender}")

# # # Loop through the list of cities and perform a request for data on each (Household)
# for year in years:
    
#     try:
#         response = requests.get(f'https://api.bjs.ojp.gov/bjs/ncvs/v2/household/{year}?format=json').json()
# #         year.append(response)
# #         income.append(response)
#         gender.append(response['householdData'][0]['hhgen'])
# #         print(f"The year is {year}")

#         print(f"The gender is {gender}")
#     except Exception:
#         print(f"Nope")
        
personal_df = pd.DataFrame(personal_data["personalData"])
household_df = pd.DataFrame(household_data['householdData'])›
# Merge the two DataFrames and display the head
merge_df = pd.merge(personal_df, household_df, on = "year")
merge_df.head()

,weight_x,year,ager,direl,ethnic1R,gender,hincome_x,hispanic,injury,locationr_x,...,hincome_y,hnumber,locationr_y,msa_y,newcrime_y,newoff_y,notify_y,popsize_y,region_y,vicservices_y
0,2624.881,2018,7,1,1,2,5,2,0,1,...,4,2,1,1,3,8,2,4,3,2
1,2624.881,2018,7,1,1,2,5,2,0,1,...,3,4,1,1,3,8,1,1,3,2
2,2624.881,2018,7,1,1,2,5,2,0,1,...,7,3,1,2,3,8,2,0,3,2
3,2624.881,2018,7,1,1,2,5,2,0,1,...,7,3,1,2,3,8,2,0,3,2
4,2624.881,2018,7,1,1,2,5,2,0,1,...,88,2,1,1,3,8,1,2,3,2


In [17]:
# Get a list of all of our columns for easy reference
merge_df.columns

Index(['weight_x', 'year_x', 'ager', 'direl', 'ethnic1R', 'gender',
       'hincome_x', 'hispanic', 'injury', 'locationr_x', 'marital2', 'msa_x',
       'newcrime_x', 'newoff_x', 'notify_x', 'popsize_x', 'race1R', 'region',
       'seriousviolent', 'treatment', 'vicservices_x', 'weapcat', 'weapon',
       'weight_y', 'year_y', 'hhage', 'hheth1R', 'hhgen', 'hhhisp', 'hhrace1R',
       'hincome_y', 'hnumber', 'locationr_y', 'msa_y', 'newcrime_y',
       'newoff_y', 'notify_y', 'popsize_y', 'vicservices_y'],
      dtype='object')

In [ ]:
# Extract "year_x", "ager", "gender", "hincome", "newoff_x", "staff_pick",
# "backers_count", and "spotlight"
reduced_kickstarter_df = df.loc[:, ["name", "goal", "pledged",
                                    "state", "country", "staff_pick", "backers_count", "spotlight"]]
reduced_kickstarter_df